In [1]:
# Packages for analysis
import pandas as pd
import numpy as np
from sklearn import svm
import random as rdm
import math

import matplotlib.pyplot as plt


# Scikit Imports
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Packages for analysis
import pandas as pd


In [2]:
J = []
Jtrain = []
Jtest = []
Jini = 0.9
Jfin = 1.1
datos = 4000
delta=(Jfin-Jini)/datos
train= []


#intervaltraining

[ J.append (Jini + (Jfin-Jini)/datos * i) for i in range (datos)]
[ Jtrain.append (J[i]) for i in range (datos) if J[i] < 0.95  and J[i] > 0.9 ]
[ Jtrain.append (J[i]) for i in range (datos) if J[i] < 1.1  and J[i] > 1.05 ]
[ Jtest.append (J[i]) for i in range (datos) if J[i] not in Jtrain]

"""Jtrainfile= 'dataJW/Jtraininter' 
np.save(Jtrainfile, Jtrain)"""

print(len(Jtrain))


"""
#randomtraining

for i in range (180):
    n=rdm.randint(0,1000)
    if len(train)<132:
        if n not in train:
            train.append(n)            

[ J.append (Jini + delta * i) for i in range (datos) ]
[ Jtrain.append(J[i]) for i in train ]
[ Jtest.append (J[i]) for i in range (datos) if J[i] not in Jtrain ]
"""

teorico_train=[]
teorico_test=[]
teorico_todo=[]

[ teorico_train.append(0) if Jtrain[i] < 1  else teorico_train.append(1) for i in range (len (Jtrain)) ]
[ teorico_test.append(0) if Jtest[i] < 1  else teorico_test.append(1) for i in range (len (Jtest)) ]
[ teorico_todo.append(0) if J[i] < 1  else teorico_todo.append(1) for i in range (len (J) ) ]


print(len(teorico_test))
print(len(teorico_train))



1998
2002
1998


In [3]:
#kernel JW

def kernelJW (J, J_p, lista):
    ker=np.empty((len(J), len(J_p)))
    for i, J1 in enumerate (J, 0):
        for j, J2 in enumerate (J_p, 0):
            ker[i,j]=1
            for a, k in enumerate (lista):
                z1 = 1 - J1 * math.cos(k)
                z2 = 1 - J2 * math.cos(k)
                y1 = J1 * math.sin(k)
                y2 = J2 * math.sin(k)
                ϵ1 = (J1 ** 2 - 2 * J1 * math.cos(k) + 1) ** (0.5)
                ϵ2 = (J2 ** 2 - 2 * J2 * math.cos(k) + 1) ** (0.5)
                v1 = y1 / ( 2 * ϵ1 * (ϵ1 + z1) )**(0.5)
                v2 = y2 / ( 2 * ϵ2 * (ϵ2 + z2) )**(0.5)
                u1 = ( (ϵ1 + z1) / (2 * ϵ1) ) ** (0.5)
                u2 = ( (ϵ2 + z2) / (2 * ϵ2) ) ** (0.5)
                ker[i,j] *= (u1 * u2 +  abs(v1) * abs(v2))
    return (ker)

In [ ]:
for N in range(100, 1000, 100):
    print(N)
    kmax = int (0.5 * N)
    #ks = list (range (0, kmax, 1) )
    ks = (2 * np.arange(0, kmax, 1) + 1) * math.pi / N
    
    """for j in range ( len(ks) ):
        ks[j] = (2 * ks[j] +1) * math.pi/N"""

    #GramMatrix para el modelo
    kernel_train = kernelJW (Jtrain, Jtrain, ks)

    """ktrainfile= 'dataJW/kerneltrain'  + str(N)
    np.save(ktrainfile, kernel_train)"""
 
    kernel_test = kernelJW (Jtest, Jtrain, ks)
    kernel_todo = kernelJW (J, Jtrain, ks)
    kernelsave = kernelJW (J, J, ks)
    """plt.imshow(kernelsave,vmin=0,vmax=1, cmap = 'gray')
    plt.show()
    plt.imshow(kernelsave2,vmin=0,vmax=1, cmap = 'gray')
    plt.show()"""
    

    #Aplicamos el modelo
    model = svm.SVC(kernel = 'precomputed' )
    model.fit (kernel_train, teorico_train)
    y_train = model.predict(kernel_train)
    y_test = model.predict(kernel_test)
    distance=model.decision_function(kernel_todo)
    supportvs=model.support_
    #print(np.shape(distance))
    

    sv=[]
    jsv=[]
    for i, n in enumerate(supportvs):
        jsv.append(Jtrain[n])


        #Cálculo del accuracy  
    print(confusion_matrix(teorico_test,y_test))
    print(classification_report(teorico_test,y_test))

            #Saving data
    Jtrainfile= 'dataJW/Jtraindiscrete3' + str(N)
    kernelfile = 'dataJW/KernelJWdiscrete3' + str(N)
    distancefile =  'dataJW/distanceJWdiscrete3'  + str(N)
    supportfile= 'dataJW/supportvsdiscrete3'  + str(N)
    data_distance = np.array([J, distance])
    np.save(Jtrainfile, Jtrain)
    np.save(kernelfile, kernelsave)
    np.save(distancefile, data_distance)
    np.save(supportfile, jsv)


100


In [ ]:
overlap = np.array([kernelsave[i, i + 1] for i in range(kernelsave.shape[0] - 1)])
#J=np.delete(J, 0)
plt.plot(J[1:], overlap)
χ =  (1 - overlap) / (delta) ** 2
#plt.plot(J[1:], χ)